In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs/fit

## 1. Preprocesamiento de los datos

### 1.1 Preparación del entorno 

Aquí lo que hago es preaprar los imports principales y las variables de entorno. Ahora están comentadas porque como te dije en el correo estaba teniendo problemas con la versión de Cuda y Keras. Ahora lo he solucionado, pero basicamente lo que pasaba es que estaba usando una versión de Keras (Keras 3) la cual no es compatible con transfomers, al final lo soluciones activando  la variable de entorno TF_USE_LEGACY_KERAS e instalando en mi env de conda tf_keras para tener Keras 2. 

La variable TF_ENABLE_XLA la desactivé porque se supone que podía ser una causa de un problema de OOM que estaba teniendo durante el entrenamiento, pero al final resultó ser un problema con las versiones de las librerías.

La variables de entorno que tienen que ver con CUDA era porque creía que mi entorno virtual de conda (el cual al final lo he tenido que meter en un WLS2 con ubuntu porque en windows estaba teniendo problemas de compatibilidad peores, estaba cogienod) estaba cogiendo la versión de CUDA que no era, porque tenía varias instaladas. Pero era más un fallo de configuración del entorno que eso.

También he añadido una sección en la que controlo si el dispositivo con el que se va a entrenar es la GPU, pongo un creciomiento progresivo en el uso de memoria para evitar sobrecarga y también un límite para evitar de nuevo el OOM.

La línea tf.config.optimizer.set_jit(False) la usaba cuando tenía desactivado el XLA para evitar así que compilase por XLA y evitar posibles problemas de rendimiento, pero ese no era el problema. 

In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
#os.environ['TF_ENABLE_XLA'] = '0'

#os.environ['CUDA_HOME']       = '/usr/local/cuda'
#os.environ['CUDA_PATH']       = '/usr/local/cuda'
#os.environ['CUDA_ROOT']       = '/usr/local/cuda'
#os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH','')

import tensorflow as tf
info = tf.sysconfig.get_build_info()
print("CUDA:",   info["cuda_version"])
print("cuDNN:",  info["cudnn_version"])

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision policy:", mixed_precision.global_policy())

import random
import numpy as np
import transformers

# Fijamos la semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Hiperparámetros
NUM_LABELS = 44  # 43 emociones + 1 sin emoción

MODEL_NAME = "monologg/kobert" #"beomi/KcELECTRA-base" 
MAX_LENGTH = 512 # Longitud máxima de las secuencias
BATCH_SIZE = 32

DROPOUT_RATE   = 0.3      
L2_REG         = 1e-5     
WEIGHT_DECAY   = 1e-5     
UNFREEZE_EPOCH = 3        

EPOCHS = 10

# Forzar el uso de la GPU y activamos el crecimiento de memoria y la limitamo para evitar el OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=14336)]
    )
    device_name = "GPU"
else:
    device_name = "CPU"

print("Dispositivo:", device_name)
    
#tf.config.optimizer.set_jit(False)

print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

2025-06-05 10:45:02.164941: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 10:45:02.400233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749113102.492993    1297 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749113102.521338    1297 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749113102.714781    1297 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

CUDA: 12.5.1
cuDNN: 9
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090 Laptop GPU, compute capability 8.9
Mixed precision policy: <Policy "mixed_float16">


/home/carlo/miniconda3/envs/tf-gpu-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dispositivo: GPU
TensorFlow version: 2.19.0
Transformers version: 4.52.1


### 1.2 Previsualización de los datos de entrenamiento, validación y test

Aquí he cargado los datasets de manera manual, pensé en hacerlo descargando directamente desde huggingface como hacen en el código de KOTE, pero ya que tenía los archivos quise probar a hacerlo así.

In [ ]:
import pandas as pd
#from datasets import load_dataset

# Cargo los datasets en local pero también podría ser desde HuggingFace como en el notebook que da KOTE: dataset = load_dataset("searle-j/kote")

train_path = "train.tsv"
val_path   = "val.tsv"    
test_path  = "test.tsv"

columns = ["id", "text", "labels"] 
df_train = pd.read_csv(train_path, sep="\t", header=None, names=columns)
df_val   = pd.read_csv(val_path,   sep="\t", header=None, names=columns)
df_test  = pd.read_csv(test_path,  sep="\t", header=None, names=columns)

print(f"Ejemplos cargados de Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

df_train.head(3)

### 1.3 Binarización de las etiquetas

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Para convertir la columna labels de string a lista de ints
def parse_labels(label_str):
    if pd.isna(label_str) or label_str == "":
        return []
    return [int(x) for x in label_str.split(",")]

train_label_lists = df_train["labels"].apply(parse_labels)
val_label_lists   = df_val["labels"].apply(parse_labels)
test_label_lists  = df_test["labels"].apply(parse_labels)

# Pasamos la lisa de etiquetas a un formato multi-hot
# (una lista de listas de etiquetas, donde cada lista tiene el mismo tamaño que el número total de etiquetas)
mlb = MultiLabelBinarizer(classes=list(range(NUM_LABELS)))
mlb.fit(train_label_lists)  # Con esto hacemos que el binarizador aprenda las etiquetas

y_train = mlb.transform(train_label_lists)
y_val   = mlb.transform(val_label_lists)
y_test  = mlb.transform(test_label_lists)

print("Tamaño de y_train:", y_train.shape)
print("Ejemplo de vector de etiquetas (multi-hot) para una muestra:\n", y_train[0])

### 1.4 Revisión de los comentarios y pasarlos a listas

In [ ]:
# Pasamos los comentarios también a listas
train_texts = df_train["text"].tolist()
val_texts   = df_val["text"].tolist()
test_texts  = df_test["text"].tolist()

print("Texto de ejemplo:", train_texts[0])
print("Etiquetas de ejemplo:", train_label_lists.iloc[0])
print("Vector multi-hot:", y_train[0])

### 1.5 Definición del tokenizador

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

### 1.6 Preparación de los datasets para Tensorflow

El tamaño de cada batch lo he puesto como último experimento en 32, no sé si 64 me daría OOM, pero como he estado teniendo los problemas que te he mendionado al principio he probado con 8, 16 y 32. 

También he optado por usar shuffle, padding y prefetch en el pipeline de datos para maximizar la eficiencia y la calidad del entrenamiento. El shuffle (con un buffer igual al tamaño del conjunto de train) garantiza que cada época reciba los ejemplos en un orden distinto, evitando correlaciones espurias y reduciendo el riesgo de sobreajuste. El padding dinámico, aplicado en cada batch, homogeneiza las secuencias a la longitud máxima del batch en lugar de usar un tamaño fijo global, lo que es más eficiente al rellenar solo lo estrictamente necesario. Finalmente, el prefetch solapa la preparación de los siguientes batches con la ejecución del modelo, de modo que mientras el GPU entrena con un batch, el pipeline ya está leyendo y procesando el siguiente, eliminando cuellos de botella y mejorando el rendimiento en general.

In [ ]:
# Con esta función generamos los datasets a partir de los textos y etiquetas
def data_generator(texts, labels):
    for text, label in zip(texts, labels):
        # Tokenizamos el texto (sin padding fijo, lo hará el batch)
        enc = tokenizer(text, truncation=True, max_length=MAX_LENGTH, padding=False)
        input_ids = enc["input_ids"] 
        attention_mask = enc["attention_mask"] # La máscara de atención sirve para indicar qué tokens son padding y cuáles no
        yield (input_ids, attention_mask), label

# Definimos los tipos y formas de salida para el dataset necesario para el método from_generator
output_types = ((tf.int32, tf.int32), tf.int32)
output_shapes = ((tf.TensorShape([None]), tf.TensorShape([None])), tf.TensorShape([NUM_LABELS])) 

# Creamos los Dataset para train, val y test
train_ds = tf.data.Dataset.from_generator(lambda: data_generator(train_texts, y_train), output_types=output_types, output_shapes=output_shapes)
val_ds   = tf.data.Dataset.from_generator(lambda: data_generator(val_texts, y_val), output_types=output_types, output_shapes=output_shapes)
test_ds  = tf.data.Dataset.from_generator(lambda: data_generator(test_texts, y_test), output_types=output_types, output_shapes=output_shapes)

# Aplicamos shuffle, batch con padding y prefetch para optimizar el rendimiento
train_ds = train_ds.shuffle(buffer_size=len(train_texts), seed=SEED, reshuffle_each_iteration=True) # (shuffle y el padding se aplican al dataset, no a los generadores)
# Padding None para input_ids y attention_mask para que se ajusten a la longitud máxima del batch 
train_ds = train_ds.padded_batch(BATCH_SIZE, padded_shapes=(([None], [None]), [NUM_LABELS]), padding_values=((tokenizer.pad_token_id, 0), 0))
# AUTOTUNE permite que TensorFlow ajuste automáticamente el número de threads para optimizar el rendimiento
train_ds = train_ds.prefetch(tf.data.AUTOTUNE) # Esto permite que el dataset se cargue en paralelo mientras el modelo entrena

# Para el val y test no es necesario hacer shuffle ya que no se entrena con ellos 
val_ds = val_ds.padded_batch(BATCH_SIZE, padded_shapes=(([None], [None]), [NUM_LABELS]), padding_values=((tokenizer.pad_token_id, 0), 0))
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.padded_batch(BATCH_SIZE, padded_shapes=(([None], [None]), [NUM_LABELS]), padding_values=((tokenizer.pad_token_id, 0), 0))
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


### 1.7 Inspección de un batch de ejemplo del conjunto de train

In [ ]:
for batch in train_ds.take(1): 
    (input_ids_batch, mask_batch), labels_batch = batch
    print("Shape input_ids:", input_ids_batch.shape)
    print("Shape attention_mask:", mask_batch.shape)
    print("Shape labels:", labels_batch.shape)
    print("Ejemplo IDs primera muestra:", input_ids_batch[0, :10].numpy())
    print("Ejemplo mask primera muestra:", mask_batch[0, :10].numpy())
    print("Etiquetas primera muestra:", labels_batch[0].numpy())

## 2. Definición del modelo 

### 2.1 Carga del modelo preentrenado de transformer

In [ ]:
from transformers import TFAutoModel, AutoConfig

config = AutoConfig.from_pretrained(MODEL_NAME)
transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
hidden_size = config.hidden_size 
print(f"Modelo base '{MODEL_NAME}' cargado. Tamaño del hidden state: {hidden_size}")

### 2.2 Pooling de Representaciones y Clasificación

He optado por aplicar una capa de pooling para reducir la salida 3D del Transformer (batch, seq_len, hidden_size) a un vector 2D (batch, hidden_size) que sirva de entrada a la capa de clasificación. Y he implementado 2 posibilidades: CLS o Mean. Basicamente CLS es rápida y está optimizada para clasificación, mientras que Mean puede capturar mejor contextos largos. 

Una vez obtenido el vector de salida de la capade pooling, aplico una capa dropout para regularizar y, finalmente, una capa densa con activación sigmoide ya que se trata de un problema de clasificación multietiqueta.

In [ ]:
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras.layers import Dropout, Dense

# Entrada del modelo: los ids y la máscara de atención
input_ids = Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# Salida del modelo
# outputs = transformer_model([input_ids, attention_mask])
outputs = transformer_model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    training=False
)
sequence_output = outputs.last_hidden_state 

# Capa de pooling
POOLING_STRATEGY = "cls"  # puede ser cls o mean

if POOLING_STRATEGY == "cls":
    # Tomamos el embedding del primer token [CLS] de cada secuencia
    pooled_output = sequence_output[:, 0, :] 
elif POOLING_STRATEGY == "mean":
    # Calculamos la media de los embeddings reales (ignorando padding)
    mask_expanded = tf.cast(tf.expand_dims(attention_mask, -1), tf.float32)  # shape [batch, seq_len, 1]
    # Multiplicar cada token embedding por la máscara (0 para pads)
    masked_output = sequence_output * mask_expanded
    # Sumar en eje de secuencia y dividir por la suma de la máscara (número de tokens reales)
    sum_embeddings = tf.reduce_sum(masked_output, axis=1)  # [batch, hidden_size]
    num_tokens = tf.reduce_sum(mask_expanded, axis=1)      # [batch, 1]
    pooled_output = sum_embeddings / (num_tokens + 1e-10)  # evitar la indeterminación

# Aplicamos el dropout
x = Dropout(DROPOUT_RATE)(pooled_output)

# Aplicamos una capa densa para reducir la dimensionalidad y añadir no linealidad
x = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(L2_REG))(x)

# Aplicamos otro dropout para evitar el sobreajuste
x = Dropout(DROPOUT_RATE)(x)

# Capa densa de clasificación con activación sigmoide (multietiqueta)
output_logits = Dense(NUM_LABELS, activation="sigmoid", kernel_regularizer=regularizers.l2(L2_REG))(x)

# Definimos el modelo completo
model = Model(inputs=[input_ids, attention_mask], outputs=output_logits)

# Visualizamos el modelo
model.summary()

### 2.3 Compilación del modelo

Aquí añadí una línea para limpiar la caché de la sesión para ver si era eso lo que estaba causando el OOM, la he comentado porque el problema no venía de ahí

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers.experimental import AdamW

# Limpiamos la sesión de Keras para evitar el OOM
# tf.keras.backend.clear_session()

# Compilamos el modelo con optimizador, función de pérdidas y métricas para la evaluación
"""
model.compile(optimizer=Adam(learning_rate=2e-5),
              loss=BinaryCrossentropy(),
              metrics=[AUC(name="AUC", multi_label=True)])
"""

# Compilamos el modelo con AdamW, que es una variante de Adam con decaimiento de peso
optimizer = AdamW(learning_rate=2e-5, weight_decay=WEIGHT_DECAY)
optimizer_ft = AdamW(learning_rate=2e-5, weight_decay=WEIGHT_DECAY)

## 3. Entrenamiento

### 3.1 Guardado del modelo y callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import re
from datetime import datetime

# Directorio para guardar el mejor modelo
#checkpoint_path = "best_model.h5"
clean_name = re.sub(r'[^A-Za-z0-9._-]', '_', MODEL_NAME)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_path = f"{clean_name}_{timestamp}.h5"

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1), # Así evitamos el overfitting
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1), # Nos quedamos con el mejor modelo
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, verbose=1), # Reducimos el learning rate si no mejora la val_loss
    TensorBoard(log_dir="logs/fit", histogram_freq=1),  # guarda logs para TensorBoard
]

### 3.2 Entrenamiento

In [ ]:
# Bloque de preentrenamiento: congelamos el modelo base y entrenamos solo la cabeza
transformer_model.trainable = False # Congelamos el modelo base para el preentrenamiento
model.compile(
    optimizer=optimizer,
    loss=BinaryCrossentropy(label_smoothing=0.1),
    metrics=[AUC(name="AUC", multi_label=True)]
)
history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=UNFREEZE_EPOCH-1,
    callbacks=callbacks
)

In [ ]:
# Bloque de fine-tuning: descongelamos el modelo base y entrenamos todo junto
transformer_model.trainable = True # Descongelamos el modelo base para el fine-tuning
# Hay que recompilar para que Keras vea los nuevos trainable flags
model.compile(
    optimizer=optimizer_ft,
    loss=BinaryCrossentropy(label_smoothing=0.1),
    metrics=[AUC(name="AUC", multi_label=True)]
)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=UNFREEZE_EPOCH-1,
    epochs=EPOCHS,
    callbacks=callbacks
)

## 4. EVALUACIÓN

### 4.1 Métricas de evaluación sobre el conjunto de test

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Obtenemos las predicciones del modelo sobre el conjunto de test
y_prob = model.predict(test_ds) 
y_pred = (y_prob >= 0.5).astype(int)

y_true = y_test # Los valores reales de las etiquetas en formato multi-hot que hemos creado antes
#print("Predicciones del modelo (probabilidades):\n", y_prob[:5])
#print("Predicciones del modelo (clases):\n", y_pred[:5])
#print("Etiquetas reales (multi-hot):\n", y_true[:5])

# Calculamos las métricas globales
precision_macro  = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall_macro     = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1_macro         = f1_score(y_true, y_pred, average='macro', zero_division=0)
precision_weighted = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall_weighted    = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_weighted        = f1_score(y_true, y_pred, average='weighted', zero_division=0)
auc_macro        = roc_auc_score(y_true, y_prob, average='macro')
auc_weighted     = roc_auc_score(y_true, y_prob, average='weighted')

print(f"Precisión macro:  {precision_macro:.4f}")
print(f"Recall macro:      {recall_macro:.4f}")
print(f"F1-score macro:    {f1_macro:.4f}")
print(f"Precisión weighted: {precision_weighted:.4f}")
print(f"Recall weighted:    {recall_weighted:.4f}")
print(f"F1-score weighted:  {f1_weighted:.4f}")
print(f"AUC macro:         {auc_macro:.4f}")
print(f"AUC weighted:      {auc_weighted:.4f}")

In [ ]:
# Reporte de clasificación por etiqueta
report = classification_report(y_true, y_pred, zero_division=0, output_dict=False)
print(report)

### 4.2 Curvas de aprendizaje

In [ ]:
import matplotlib.pyplot as plt

"""
# Extraemos el historial del entrenamiento y de validación
epochs = range(1, len(history.history['loss'])+1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_auc = history.history.get('AUC')  
val_auc = history.history.get('val_AUC')

# Creamos la grafica de las pérdidas
plt.figure(figsize=(8,4))
plt.plot(epochs, train_loss, 'o-', label='Training Loss')
plt.plot(epochs, val_loss, 'o--', label='Validation Loss')
plt.title('Curva de pérdida')
plt.xlabel('Epoch')
plt.ylabel('Binary Crossentropy Loss')
plt.legend()
plt.show()

# Creamos la grafica de AUC si está disponible
if train_auc:
    plt.figure(figsize=(8,4))
    plt.plot(epochs, train_auc, 'o-', label='Training AUC')
    plt.plot(epochs, val_auc, 'o--', label='Validation AUC')
    plt.title('Curva de AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    plt.show()
"""

loss = history_head.history['loss'] + history_finetune.history['loss']
val_loss = history_head.history['val_loss']  + history_finetune.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(8,5))
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.axvline(UNFREEZE_EPOCH-1, color='gray', linestyle='--', label='Start Fine-tuning')
plt.title('Loss over epochs (head-only + fine-tuning)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

auc = history_head.history['AUC'] + history_finetune.history['AUC']
val_auc = history_head.history['val_AUC']  + history_finetune.history['val_AUC']

plt.figure(figsize=(8,5))
plt.plot(epochs, auc, 'bo-', label='Training AUC')
plt.plot(epochs, val_auc, 'ro-', label='Validation AUC')
plt.axvline(UNFREEZE_EPOCH-1, color='gray', linestyle='--', label='Start Fine-tuning')
plt.title('AUC over epochs (head-only + fine-tuning)')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.show()


### 4.3 Matriz de confusión binaria por emoción

In [ ]:
from sklearn.metrics import confusion_matrix

for label_idx in range(NUM_LABELS):
    cm = confusion_matrix(y_true[:, label_idx], y_pred[:, label_idx], labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    label_name = str(label_idx)  
    print(f"Emoción {label_name}: TP={tp}, FP={fp}, FN={fn}, TN={tn}")

Aquí mapeo el id de las etiquetas al nombre real de la emoción. El problema es que no estoy seguro de si el mapeo es correcto, ya que cojo el nombre de las etiquetas del github de KOTE: https://github.com/searle-j/KOTE entonces no sé si el orden es el correcto.

In [ ]:
from sklearn.metrics import confusion_matrix

LABELS = [
    '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경',
    '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적',
    '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움',
    '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음',
    '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움',
    '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰'
]

for label_idx, label_name in enumerate(LABELS):
    cm = confusion_matrix(
        y_true[:, label_idx],
        y_pred[:, label_idx],
        labels=[0, 1]
    )
    tn, fp, fn, tp = cm.ravel()
    print(f"Emoción «{label_name}»: TP={tp}, FP={fp}, FN={fn}, TN={tn}")

### 4.4 Predicciones fallidas

In [ ]:
# Identificamos los índices de predicciones con errores
error_indices = []
for i in range(len(y_test)):
    true_set = np.where(y_true[i] == 1)[0].tolist()
    pred_set = np.where(y_pred[i] == 1)[0].tolist()
    true_set = [str(x) for x in true_set]
    pred_set = [str(x) for x in pred_set]
    if true_set != pred_set: # Comparamos las etiquetas reales y predichas
        error_indices.append((i, true_set, pred_set))

print(f"Número de ejemplos de prueba con error en al menos una etiqueta: {len(error_indices)}")

# Mostramos ejemplos al azar
for idx, true_labels, pred_labels in random.sample(error_indices, 5):
    text = test_texts[idx]
    print(f"\nComentario: {text}")
    print(f"Etiquetas reales: {', '.join(true_labels) if true_labels else 'None'}")
    print(f"Etiquetas predichas: {', '.join(pred_labels) if pred_labels else 'None'}")